In [1]:
import sys, importlib as impL
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/vae_torch')
sys.path.insert(1,'/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery')
import vae_torch_model as vtm
import vae_torch as vt
#from data_classes import khs_dataset_v2
import vae_scripts as vs
import sae_torch as st
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import vae_utils as vu
import pandas as pd
import helperFuncs as funcH

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option("display.precision", 3)
pd.set_option('display.max_rows', None)

/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/doga/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
def get_acc_err(experiments_folder, exp_base_name,cf_int,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=1):
    ae_fold_name = os.path.join(experiments_folder, exp_base_name + str(cf_int).zfill(z_fill_int))
    ae_f_name = os.path.join(ae_fold_name, ae_f_name_base)
    vfz = np.load(ae_f_name, allow_pickle=True)
    loss_log_dict = {}
    n = 0
    loss_log_dict[data_log_key] = vfz.item().get(data_log_key)
    if loss_log_dict[data_log_key] is None:
        print("cf("+str(cf_int)+") --> loss_log_dict"+str(data_log_key)+"] is none")
        return None, None, None
    n = len(loss_log_dict[data_log_key])
    if verbose>0:
        print(str(cf_int), ', ', data_log_key, " - log is loaded with len: ", n)
    if loss_key not in loss_log_dict[data_log_key][0].keys():
        return n, None, None

    if loss_key in loss_log_dict[data_log_key][0]:
        vec_len = len(loss_log_dict[data_log_key])
        los_vec_cur = [loss_log_dict[data_log_key][l][loss_key] for l in range(0, vec_len)]
        label_str = str(cf_int) + '_' + data_log_key + '_' + loss_key
        if verbose>0:
            print(label_str, los_vec_cur[-3:], "\nmax({:4.2f}),min({:4.2f})".format(np.max(los_vec_cur), np.min(los_vec_cur)))
        return n, np.min(los_vec_cur), np.max(los_vec_cur)
    return None, None, None

In [3]:
impL.reload(vu)
experiments_folder='/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/experiments/FM/'
config_folder = '/mnt/USB_HDD_1TB/GitHub/keyhandshapediscovery/configs/'
z_fill_int=2
loss_key='reconstruction' # sparsity, bottleneck_kmeans, bottleneck_act
data_log_keys=['tr_te']  # ['tr_va', 'va', 'te']
exp_base_name='exp_conv_ae_simple_is28_cf'
ae_f_name_base='ae_ft_conv_ae_simple_is28.npy'
#cf_int_arr=[201,202,203,204,205,206,207,208,209,210,211,212,213,214,216,217,218]
#cf_int_arr=[201,204,205,207,213,214]
#cf_int_arr=[201,204,205,216,217,218]
#cf_int_arr=[220,221,222,223,224,225,226,227]
cf_int_arr=np.asarray(range(220,249), dtype=int)
df1 = pd.DataFrame(index=cf_int_arr, columns=['spMethod','spW','spR','kl_div','bact','sigm','logsm','lact','rcErr','lr','rcRed'])
df2 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
df3 = pd.DataFrame(index=cf_int_arr, columns=['n','bActMin','bActMax','bErrMin','bErrMax','kmMin','kmMax','rErrMax','rErrMin'])
df4 = pd.DataFrame(index=cf_int_arr, columns=['kl_div','bact','sigm','logsm','lr', 'n','bActMax','bErrMin','bErrDif','kmMax','rErrDif'])
for i in range(0,len(cf_int_arr)):
    cf = cf_int_arr[i]
    #load model yaml as config
    config_file = os.path.join(config_folder, 'conf_autoencoder_'+str(cf)+'.yaml')
    CONF_PARAMS_ = funcH.CustomConfigParser(config_file=config_file)
    df1.iloc[i]['spW'] = float(CONF_PARAMS_.MODEL.SPARSITY_WEIGHT)
    if df1.iloc[i]['spW'] > 0:
        df1.iloc[i]['spMethod'] = CONF_PARAMS_.MODEL.SPARSITY_ERROR
        df1.iloc[i]['spR'] = CONF_PARAMS_.MODEL.SPARSITY_REDUCTION
    else:
        df1.iloc[i]['spW'] = None
    df1.iloc[i]['kl_div'] = CONF_PARAMS_.MODEL.KL_DIV_RHO if 'kl_' in str(CONF_PARAMS_.MODEL.SPARSITY_ERROR) else None
    df4.iloc[i]['kl_div'] = df1.iloc[i]['kl_div']
    try:
        df1.iloc[i]['bact'] = (CONF_PARAMS_.MODEL.LAYERS.encoder.l04_act).replace('type: ','').replace(',dim:','')
        df4.iloc[i]['bact'] = df1.iloc[i]['bact']
    except:
        pass
    df1.iloc[i]['sigm'] = CONF_PARAMS_.MODEL.KL_SIGMOID
    df4.iloc[i]['sigm'] = df1.iloc[i]['sigm']
    df1.iloc[i]['logsm'] = CONF_PARAMS_.MODEL.KL_LOGSOFTMAX
    df4.iloc[i]['logsm'] = df1.iloc[i]['logsm']
    try:
        df1.iloc[i]['lact'] = CONF_PARAMS_.MODEL.LAYERS.decoder.l01_act.replace('type: ','')
    except:
        pass
    df1.iloc[i]['rcErr'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_FUNCTION
    df1.iloc[i]['lr'] = CONF_PARAMS_.MODEL.LEARNING_RATE
    df4.iloc[i]['lr'] = df1.iloc[i]['lr']
    df1.iloc[i]['rcRed'] = CONF_PARAMS_.MODEL.RECONSTRUCTION_ERROR_REDUCTION
    
    try:
        data_log_key = 'tr_te'
        loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
        n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['n'] = n
        df4.iloc[i]['n'] = df2.iloc[i]['n']

        df2.iloc[i]['rErrMin'] = mn
        df2.iloc[i]['rErrMax'] = mx
        df4.iloc[i]['rErrDif'] = df2.iloc[i]['rErrMax']-df2.iloc[i]['rErrMin']
            
        loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['kmMin'] = mn
        df2.iloc[i]['kmMax'] = mx

        loss_key = 'bottleneck_act' #  sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['bActMin'] = mn
        df2.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'
        df4.iloc[i]['bActMax'] = df2.iloc[i]['bActMax']

        loss_key = 'sparsity' #  
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df2.iloc[i]['bErrMin'] = mn
        df2.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'
        df4.iloc[i]['bErrMin'] = df2.iloc[i]['bErrMin']

        try:
            df4.iloc[i]['bErrDif'] = df2.iloc[i]['bErrMax']-df2.iloc[i]['bErrMin']
        except:
            df4.iloc[i]['bErrDif'] = 0
        data_log_key = 'te'
        loss_key = 'reconstruction' # reconstruction bottleneck_kmeans bottleneck_act sparsity
        n, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['n'] = n

        df3.iloc[i]['rErrMin'] = mn
        df3.iloc[i]['rErrMax'] = mx

        loss_key = 'bottleneck_kmeans' # bottleneck_act sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['kmMin'] = mn
        df3.iloc[i]['kmMax'] = mx
        df4.iloc[i]['kmMax'] = df2.iloc[i]['kmMax']

        loss_key = 'bottleneck_act' #  sparsity
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['bActMin'] = mn
        df3.iloc[i]['bActMax'] = mx if mx!=mn else '_n*c_'

        loss_key = 'sparsity' #  
        _, mn, mx = get_acc_err(experiments_folder, exp_base_name,cf,ae_f_name_base,data_log_key,loss_key,z_fill_int=3,verbose=0)
        df3.iloc[i]['bErrMin'] = mn
        df3.iloc[i]['bErrMax'] = mx if mx!=mn else '_n*c_'
    except:
        print("no data available for d3 - ", cf)

In [4]:
pd.set_option("display.precision", 2)
print('experiments explanations')
print(df1)

experiments explanations
          spMethod    spW        spR  kl_div  bact   sigm  logsm lact rcErr      lr rcRed
220  kl_divergence  0.001  batchmean  0.0001  ReLu   True   True  NaN   MSE    0.01  mean
221  kl_divergence  0.001  batchmean  0.0001  ReLu   True  False  NaN   MSE    0.01  mean
222  kl_divergence  0.001  batchmean  0.0001  ReLu  False  False  NaN   MSE    0.01  mean
223  kl_divergence  0.001  batchmean  0.0001  ReLu  False   True  NaN   MSE    0.01  mean
224  kl_divergence  0.001  batchmean  0.0001   NaN   True   True  NaN   MSE    0.01  mean
225  kl_divergence  0.001  batchmean  0.0001   NaN   True  False  NaN   MSE    0.01  mean
226  kl_divergence  0.001  batchmean  0.0001   NaN  False  False  NaN   MSE    0.01  mean
227  kl_divergence  0.001  batchmean  0.0001   NaN  False   True  NaN   MSE    0.01  mean
228  kl_divergence  0.001  batchmean  0.0001  ReLu   True   True  NaN   MSE   0.001  mean
229  kl_divergence  0.001  batchmean  0.0001  ReLu   True  False  NaN   MSE

In [5]:
print('tr_te dataset results')
pd.set_option("display.precision", 5)
print(df2)

tr_te dataset results
       n bActMin bActMax     bErrMin     bErrMax   kmMin   kmMax rErrMax rErrMin
220    5      10   _n*c_    -0.32354    -0.23029      10  12.312  50.441  48.848
221    5      10   _n*c_      1870.3      1871.8     8.5      10  50.349   47.77
222    5      10   _n*c_      1657.3      1664.6      10      10  50.513  49.032
223    5      10   _n*c_     -5.1343     -3.7721   9.405      10  50.343  48.716
224    5      10   _n*c_    -0.39536    -0.37654      10  13.383  50.938  48.591
225    5      10   _n*c_      1855.1        1858      10      10  50.528  48.689
226    5      10   _n*c_      1553.8      1568.6      10      10  51.014  48.405
227    5      10   _n*c_     -6.7183     -5.8667      10      10  50.648  48.296
228    5      10   _n*c_     -8.0924     -7.4322  16.202  20.808  39.162  38.195
229    5      10   _n*c_      1909.7      1917.5  16.725  20.722  39.092   38.09
230    5      10   _n*c_      1838.4      1921.1  16.733   20.76  39.351  38.005
231   

In [6]:
print('te dataset results')
print(df3)

te dataset results
       n bActMin bActMax     bErrMin     bErrMax  kmMin  kmMax rErrMax rErrMin
220    5      10   _n*c_   -0.054498    -0.03879     10  13.23  8.4548  8.1881
221    5      10   _n*c_      315.04      315.29     10  10.83  8.4399  8.0067
222    5      10   _n*c_      279.16      280.39     10     10   8.466   8.218
223    5      10   _n*c_    -0.86485    -0.63538   8.95     10  8.4391  8.1659
224    5      10   _n*c_   -0.066596   -0.063425     10  13.14  8.5389  8.1439
225    5      10   _n*c_      312.47      312.97     10     10  8.4703   8.163
226    5      10   _n*c_      261.73      264.21     10     10  8.5542   8.113
227    5      10   _n*c_     -1.1316     -0.9882     10     10  8.4889  8.0961
228    5      10   _n*c_     -1.3647     -1.2535  15.52  20.41  6.5716  6.4089
229    5      10   _n*c_      321.68      323.01  17.81  20.29   6.561  6.3919
230    5      10   _n*c_      309.67      323.61   17.1   21.3  6.6036  6.3761
231    5      10   _n*c_ -1.8278e

In [7]:
print('picky results')
print(df4)

picky results
     kl_div  bact   sigm  logsm      lr    n bActMax     bErrMin     bErrDif   kmMax  rErrDif
220  0.0001  ReLu   True   True    0.01    5   _n*c_    -0.32354     0.09325  12.312   1.5933
221  0.0001  ReLu   True  False    0.01    5   _n*c_      1870.3      1.5248      10   2.5791
222  0.0001  ReLu  False  False    0.01    5   _n*c_      1657.3      7.3107      10   1.4812
223  0.0001  ReLu  False   True    0.01    5   _n*c_     -5.1343      1.3623      10   1.6265
224  0.0001   NaN   True   True    0.01    5   _n*c_    -0.39536    0.018824  13.383   2.3469
225  0.0001   NaN   True  False    0.01    5   _n*c_      1855.1      2.9665      10   1.8385
226  0.0001   NaN  False  False    0.01    5   _n*c_      1553.8      14.737      10   2.6093
227  0.0001   NaN  False   True    0.01    5   _n*c_     -6.7183     0.85159      10   2.3518
228  0.0001  ReLu   True   True   0.001    5   _n*c_     -8.0924     0.66022  20.808  0.96679
229  0.0001  ReLu   True  False   0.001    5  